In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type = "post"
padding_type = "post"
oov_token = "<OOV>"
training_size = 20000

In [3]:
with open("./sarcasm.json", "r") as f:
    datastore = json.load(f)
    
sentences = []
labels = []
for item in datastore:
    sentences.append(item["headline"])
    labels.append(item["is_sarcastic"])

In [4]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [5]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, padding="post", maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, padding="post", maxlen=max_length, truncating=trunc_type)

In [6]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 24)                38424     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 198,449
Trainable params: 198,449
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 30
history = model.fit(training_padded,
                    training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels))

Epoch 1/30
625/625 [==============================] - 2s 3ms/step - loss: 0.4470 - accuracy: 0.7673 - val_loss: 0.3189 - val_accuracy: 0.8614
Epoch 2/30
625/625 [==============================] - 2s 3ms/step - loss: 0.1975 - accuracy: 0.9246 - val_loss: 0.3201 - val_accuracy: 0.8644
Epoch 3/30
625/625 [==============================] - 2s 3ms/step - loss: 0.1039 - accuracy: 0.9651 - val_loss: 0.3750 - val_accuracy: 0.8599
Epoch 4/30
625/625 [==============================] - 2s 3ms/step - loss: 0.0486 - accuracy: 0.9859 - val_loss: 0.4709 - val_accuracy: 0.8515
Epoch 5/30
625/625 [==============================] - 2s 3ms/step - loss: 0.0208 - accuracy: 0.9960 - val_loss: 0.5205 - val_accuracy: 0.8486
Epoch 6/30
625/625 [==============================] - 2s 3ms/step - loss: 0.0097 - accuracy: 0.9988 - val_loss: 0.6040 - val_accuracy: 0.8484
Epoch 7/30
625/625 [==============================] - 2s 3ms/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 0.6358 - val_accuracy: 0.8492
Epoch 